In [185]:
# pip install folium
# запустите, если у вас не работают карты

import pandas as pd
import numpy as np
import folium
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import warnings


In [186]:
features = pd.read_csv("/Users/artemshevchenko/Desktop/ВК/features.csv", delimiter=",")
test_data = pd.read_csv("/Users/artemshevchenko/Desktop/ВК/test.csv")
train_data = pd.read_csv('/Users/artemshevchenko/Desktop/ВК/train.csv')


## Отбор коррелированных и зависимых признаков: 

Для начала, составим матрицу корреляции признаков и отбросим высоко-коррелированные. Ниже приложен код для иллюстрации мотивации выбора границы корреляции.


In [187]:
warnings.filterwarnings('ignore')

corr_matrix = features.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

threshold = 0.55
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
features.drop(columns=to_drop, inplace=True)

first_columns = features.columns[:2]
other_columns = range(0, len(features.columns) - 2)
new_columns = list(first_columns) + list(other_columns)

features.columns = new_columns
features

warnings.filterwarnings('default')

Рассмотрим распределение объектов ритейла. Сразу заметим, что данные по России, а объекты в основном сконцентрированы вокруг крупных российских городов - Москвы, Санкт-Петербурга, Владивостока, Казани, Новосибирска, Омска, Ростова и прочих. 

In [188]:
mymap = folium.Map(location=[55.755825, 37.617298], zoom_start=5)

for index, row in features.iterrows():
    lat = row['lat']
    lon = row['lon']
    folium.Marker(location=[lat, lon], tooltip='Точка').add_to(mymap)

# mymap

## Генерация признаков:

Так как мне неизвестно ничего по дополнительным признакам, лежащим в features, а единственные признаки, которые содержат известную мне характеристику - долгота и ширина, попробуем сгенерировать новые на их основе. В частности, мы можем считать Евклидово расстояние от каждого объекта до какой-то важной для ритейла точки - в случае с Россией это крупные мегаполисы.

В качестве примера, сгенерируем признак, заключащющийся в расстояния координат от Москвы, и другой в расстоянии от Питера.
(для простоты возьмём координаты Московского Кремля - 55.751999,37.617734 и центра СПб - 59.9386, 30.3141).
Хоть далее мы и будем обучать кластеризацию на координаты и алгоритм наверняка сам заметит, что эти города скапливают вокруг себя множество точек ритейла и добавит их в features, то эта часть нужна только в качестве примера.

Расстояния, будут иметь достаточно большую ошибку, если мы будем считать их классическими метриками. Пропишем функцию для подсчёта гаверсинусового расстояния между двумя точками на сфере:

In [189]:
lat_moscow, lon_moscow = 55.751999, 37.617734
lat_spb, lon_spb = 59.9386, 30.3141

def haversine(lat1, lon1, lat2, lon2):
    R = 6371

    lat1_rad, lon1_rad = np.radians(lat1), np.radians(lon1)
    lat2_rad, lon2_rad = np.radians(lat2), np.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = np.sin(dlat / 2) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))

    distance = R * c
    return distance


Воспользуемся найденной локальной оптимальной точкой минимума функции g(n_clusters_i) = mae_i в файле features.ipynb для обучения основной модели.

In [190]:
n_clusters = 6

X = features[['lat', 'lon']]

kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X)

features['cluster'] = kmeans.labels_
cluster_centers = kmeans.cluster_centers_

for i in range(n_clusters):
    cluster_lat, cluster_lon = cluster_centers[i]
    # print(f"Кластер {i+1}: средняя широта {cluster_lat}, средняя долгота {cluster_lon}") - можно посмотреть координаы центров кластеров


In [191]:
map_clusters = folium.Map(location=[55.755825, 37.617298], zoom_start=5)

for i in range(n_clusters):
    cluster_lat, cluster_lon = cluster_centers[i]
    features[f'distance_to_cluster{i}'] = features.apply(lambda row: haversine(row['lat'], row['lon'], cluster_lat, cluster_lon), axis=1)
    folium.Marker(location=[cluster_lat, cluster_lon], tooltip=f'Кластер {i+1}').add_to(map_clusters)
    
# map_clusters - при необходмиости можно посмотреть карту с кластерами


Соеденим файл features.ipynb, дополненный сгененрированными признаками, с тестовой и тренировочной выборкой. Так как id в датасетах не соответствует друг другу и идут не по порядку, мы воспользуемся следующей идеей - из-за погрешности в типах координаты в разных датасетах могут незначительно отличаться, так что мы зафиксируем некоторый \varepsilion такой, что если координаты train\test_data_i будут абсолютно отличаться от координат объекта features_j менее, чем на это число, то мы будем считать, что это один и тот же объект. Границы threshold определяются как минимальные для сохранения всех исходных данных.

 Убедимся, что таким образом действительно происходит биекция и мы нигде не теряем данные:

In [192]:
train_data = pd.read_csv('/Users/artemshevchenko/Desktop/ВК/train.csv')

threshold_distance = 0.2

distances = cdist(train_data[['lat', 'lon']], features[['lat', 'lon']], metric='euclidean')
nearest_indices = np.argmin(distances, axis=1)
nearest_distances = np.min(distances, axis=1)

merged_data_train = train_data.copy()
merged_data_train['nearest_index'] = nearest_indices
merged_data_train['nearest_distance'] = nearest_distances
merged_data_train = merged_data_train[merged_data_train['nearest_distance'] < threshold_distance]
merged_data_train = pd.merge(merged_data_train, features, left_on='nearest_index', right_index=True)
merged_data_train = merged_data_train.drop(['nearest_index', 'nearest_distance'], axis=1)

In [193]:
test_data = pd.read_csv("/Users/artemshevchenko/Desktop/ВК/test.csv")

threshold_distance = 0.55

distances = cdist(test_data[['lat', 'lon']], features[['lat', 'lon']], metric='euclidean')
nearest_indices = np.argmin(distances, axis=1)
nearest_distances = np.min(distances, axis=1)

merged_data_test = test_data.copy()
merged_data_test['nearest_index'] = nearest_indices
merged_data_test['nearest_distance'] = nearest_distances
merged_data_test = merged_data_test[merged_data_test['nearest_distance'] < threshold_distance]
merged_data_test = pd.merge(merged_data_test, features, left_on='nearest_index', right_index=True)
merged_data_test = merged_data_test.drop(['nearest_index', 'nearest_distance'], axis=1)


Воспользуемся алгоритмом случайного леса для обучения модели. Поделим тренировочную выборку на части и попробуем сначала проанализировать результаты алгоритма с возможностью проверки реальных ззначений score:

In [194]:
warnings.filterwarnings('ignore')

X = merged_data_train.drop(columns=['score'], axis=1)
y = merged_data_train['score']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)

mae = mean_absolute_error(y_test, predictions)
print("Mean Absolute Error:", mae)

warnings.filterwarnings('default')

Mean Absolute Error: 0.06126545126350084


Ошибка достаточно хорошая - можем прогнать модель через тестовые данные и выполнить задачу.

In [195]:
merged_data_train

,id,lat_x,lon_x,score,lat_y,lon_y,0,1,2,3,...,20,21,22,cluster,distance_to_cluster0,distance_to_cluster1,distance_to_cluster2,distance_to_cluster3,distance_to_cluster4,distance_to_cluster5
0,0,56.228300,43.945535,0.080523,56.228936,43.945563,0.142355,0.291666,0.112751,0.010147,...,1.0,0.454545,0.051389,4,946.277143,375.714220,2395.375457,899.701783,290.771668,1175.696712
1,1,56.834244,53.141543,0.104424,56.834461,53.142202,0.094093,0.437761,0.106110,0.056220,...,1.0,0.454545,0.055459,4,383.759443,943.206334,1828.373497,1369.599339,348.751166,1472.637583
2,2,45.042299,41.990170,0.067615,45.042112,41.989783,0.151723,0.388125,0.178486,0.020507,...,1.0,0.454545,0.077430,5,1725.932406,1185.849973,3045.531823,1825.596132,1221.106389,95.670085
3,3,59.849408,30.387762,0.088038,59.848909,30.387232,0.423663,0.570066,0.157089,0.032190,...,1.0,0.454545,0.050685,3,1736.078711,672.848492,3086.054768,8.216297,1176.930648,1724.248252
4,4,59.839643,30.304308,0.099686,59.839632,30.304086,0.489140,0.527301,0.118082,0.118855,...,1.0,0.454545,0.059898,3,1740.705301,675.508023,3090.812095,5.686096,1181.043355,1725.667373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3077,3077,51.500685,45.951897,0.099876,51.500685,45.951860,0.033270,0.172172,0.097444,0.054565,...,1.0,0.454545,0.055855,4,1012.574722,677.232974,2433.695828,1348.909486,446.552620,719.774546
3079,3079,55.674584,37.279505,0.531557,55.673937,37.279167,0.628328,0.421714,0.142870,0.027980,...,1.0,0.454545,0.063838,1,1365.701857,59.239577,2812.047844,625.856965,693.438635,1136.541093
3080,3080,55.784909,49.188791,0.101631,55.785428,49.188083,0.193135,0.452026,0.163252,0.025054,...,1.0,0.454545,0.051499,4,628.054589,696.710890,2086.844167,1204.391119,81.233515,1239.473881
3081,3081,55.118828,61.462996,0.125265,55.118176,61.462914,0.127627,0.465737,0.158460,0.033000,...,1.0,0.454545,0.070415,0,178.674001,1473.220575,1335.265543,1917.878387,836.229913,1751.554111


In [196]:
warnings.filterwarnings('ignore')

train_score = merged_data_train["score"]
merged_data_train = merged_data_train.drop(["score"], axis=1)

model = RandomForestRegressor(random_state=42)
model.fit(merged_data_train, train_score)

predictions = model.predict(merged_data_test)

res = pd.DataFrame(predictions)
res["id"] = merged_data_test["id"]
res = res[['id', 0]]

warnings.filterwarnings('default')

In [197]:
# на случай, если на вашем ядре не работает следующая клетка и импорт solution.py: просто раскомментируйте этот кусок
# и запустите последний блок для сохранения предсказаний в файл

# def save_predictions_to_csv(model, test_data, features, output_path):
    
#     distances = cdist(test_data[['lat', 'lon']], features[['lat', 'lon']], metric='euclidean')
#     nearest_indices = np.argmin(distances, axis=1)
#     nearest_distances = np.min(distances, axis=1)

#     merged_data_test = test_data.copy()
#     merged_data_test['nearest_index'] = nearest_indices
#     merged_data_test['nearest_distance'] = nearest_distances
#     merged_data_test = merged_data_test[merged_data_test['nearest_distance'] < 0.5]
#     merged_data_test = pd.merge(merged_data_test, features, left_on='nearest_index', right_index=True)
#     merged_data_test = merged_data_test.drop(['nearest_index', 'nearest_distance'], axis=1)

#     predictions = model.predict(merged_data_test)
#     res = pd.DataFrame(predictions)
#     res["id"] = merged_data_test["id"]
#     res = res[['id', 0]]
#     res = res.rename(columns={0: "score"})

#     res.to_csv(output_path, index=False)

In [198]:
save_predictions_to_csv(model, test_data, features, "/Users/artemshevchenko/Desktop/ВК/submission.csv")

/Users/artemshevchenko/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
